<a href="https://colab.research.google.com/github/sjsje101/titanic/blob/main/29.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#pipelines chain multiple steps together so that the output of each steps is used as input to the next step
#pipelines make it easy to apply the same preprocessing to train and test!
#for eg i have a data which has a missing value,categorical varibale ,encoding and then we will train;
#so by using pipelining we can chain these steps so it is easy to train model



In [ ]:
#without pipeline


In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier

In [ ]:
df=pd.read_csv('tested.csv')

In [ ]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [ ]:
#dropping the unused column so to make understanding peiplining easy
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [ ]:
df.head()
#our end goal is to predict whether the patient will be died or be saved by taking other column as input

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,34.5,0,0,7.8292,Q
1,1,3,female,47.0,1,0,7.0000,S
2,0,2,male,62.0,0,0,9.6875,Q
3,0,3,male,27.0,0,0,8.6625,S
4,1,3,female,22.0,1,1,12.2875,S


In [ ]:
#step 1
X_train,X_test,y_train,y_test=train_test_split(df.drop(columns=['Survived']),df['Survived'],test_size=0.2,random_state=42)
y_test


,Survived
321,0
324,1
388,0
56,0
153,1
...,...
57,0
126,0
24,1
17,0


In [ ]:
df.isnull().sum()

,0
Survived,0
Pclass,0
Sex,0
Age,86
SibSp,0
Parch,0
Fare,1
Embarked,0


In [ ]:
#replacing the missing value  by its mean by using simple imputuer
#repacing the missing value fare too by its means
si_age=SimpleImputer()
si_fare=SimpleImputer()

X_train_age=si_age.fit_transform(X_train[['Age']])
X_train_fare=si_fare.fit_transform(X_train[['Fare']])

X_test_age=si_age.transform(X_test[['Age']])
X_test_fare=si_fare.transform(X_test[['Fare']])

In [ ]:
#Applying one hot encoding on sex and embarked column
#if in future we get any new category ignore it
ohe_sex=OneHotEncoder(sparse_output=False,handle_unknown='ignore')
ohe_embarked=OneHotEncoder(sparse_output=False,handle_unknown='ignore')

X_train_sex=ohe_sex.fit_transform(X_train[['Sex']])
X_train_embarked=ohe_embarked.fit_transform(X_train[['Embarked']])

X_test_sex=ohe_sex.transform(X_test[['Sex']])
X_test_embarked=ohe_embarked.transform(X_test[['Embarked']])

In [ ]:
#dropping the column of which we have a arrAY
X_train_rem=X_train.drop(columns=['Sex','Age','Embarked','Fare'])
X_test_rem=X_test.drop(columns=['Sex','Age','Embarked','Fare'])

In [ ]:
X_train_transformed=np.concatenate((X_train_rem,X_train_age,X_train_sex,X_train_fare,X_train_embarked),axis=1)
X_test_transformed=np.concatenate((X_test_rem,X_test_age,X_test_sex,X_test_fare,X_test_embarked),axis=1)

In [ ]:
X_test_transformed.shape

(84, 10)

In [ ]:
clf=DecisionTreeClassifier()
clf.fit(X_train_transformed,y_train)

DecisionTreeClassifier()

In [ ]:
y_pred=clf.predict(X_test_transformed)
y_pred

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

1.0

In [ ]:
#lets say we have to deploy this model in a website our model will take input and and tell whether the
#passenger will be dead or saved


In [ ]:
#for web development and exporting ml code in dev we use pickle lib
import pickle

In [ ]:
pickle.dump(ohe_sex,open('ohe_sex.pkl','wb'))#sedning these for ohe in the user data
pickle.dump(ohe_embarked,open('ohe_embarked.pkl','wb'))
pickle.dump(clf,open('clf.pkl','wb'))# exporting clf obj at the given loc

In [ ]:
#now using model

In [ ]:
import pickle
import numpy as np

In [ ]:
ohe_sex=pickle.load(open('ohe_sex.pkl','rb'))
ohe_embarked=pickle.load(open('ohe_embarked.pkl','rb'))
clf=pickle.load(open('clf.pkl','rb'))
#unloading the data

In [ ]:
#assume user input
test_input=np.array([2,'male',31.0,0,0,10.5,'S'],dtype=object).reshape(1,7)

In [ ]:
test_input_sex=ohe_sex.transform(test_input[:,1].reshape(1,1))

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [ ]:
test_input_sex

array([[0., 1.]])

In [ ]:
test_input_embarked=ohe_embarked.transform(test_input[:,-1].reshape(1,1))

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [ ]:
test_input_age=test_input[:,2].reshape(1,1)
test_input_fare=test_input[:,5].reshape(1,1)

In [ ]:
test_input_transformed=np.concatenate((test_input[:,[0,3,4]],test_input_age,test_input_sex,test_input_fare,test_input_embarked),axis=1)

In [ ]:
test_input_transformed.shape
#one passenger data is divided into 10 col

(1, 10)

In [ ]:
clf.predict(test_input_transformed)
 #shows that the passenger will die

#  --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

array([0])

In [ ]:
# as we saw that without pipeline we have to use various steps before training model
# we have to encode and apply simple imptuter
# even after training model we have to apply these in the user input too during production
#that make it so much hectic now we will use pipelining

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier


In [ ]:
df=pd.read_csv('tested.csv')

In [ ]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [ ]:
# our plan is
# as i know i have missing data in age and fare
# 1) impute missing value by using one column transformer and the output of that coln will be given to other column
#2) ohe hot encoding in next xolumn transformer and the o/p of this will be
#3) scaling -- is dont to bring every column at a similar numerical range
#4) feature selection ---selecting best 5 features
#5) decision tree



In [ ]:
# dropping the unsused column
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)


In [ ]:
#train test split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['Survived']),df['Survived'],test_size=0.2,random_state=42)
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
336,2,male,32.0,0,0,13.0000,S
31,2,male,24.0,2,0,31.5000,S
84,2,male,NaN,0,0,10.7083,Q
287,1,male,24.0,1,0,82.2667,S
317,2,male,19.0,0,0,10.5000,S


In [ ]:
 # creating first column tranformer
 #passing 2 imputuer in tuple
 #using by index cause o/p of column transformer will be numpy array and numpy array does not have name

 trf1 = ColumnTransformer([
     ('impute_age',SimpleImputer(),[2]),
     ('impute_fare',SimpleImputer(),[5])
 ],remainder='passthrough')

In [ ]:
#ohe
trf2=ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse_output=False,handle_unknown='ignore'),[1,6])
],remainder='passthrough')

In [ ]:
#scaling using min max scalar instead of standard cause we have to do feature selction
trf3=ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10))
])


In [ ]:
#feature scaling
trf4=SelectKBest(score_func=chi2,k=8)

In [ ]:
trf5=DecisionTreeClassifier()


In [ ]:
## creating piepline


In [ ]:
pipe=Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5)
])
#passing list of two tuple
#1) column name,its object

In [ ]:
# pipeline vs make pipeline
# make pipeline is simper
# alternate syntax
pipe=make_pipeline(trf1,trf2,trf3,trf4,trf5)

In [ ]:
#train now
pipe.fit(X_train,y_train)

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_fare',
                                                  SimpleImputer(), [5])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('selectkbest',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x7d451d13fa60>)),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [ ]:
y_pred=pipe.predict(X_test)

In [ ]:
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.5952380952380952

In [ ]:
# cross validation using pipeline
from sklearn.model_selection import cross_val_score
cross_val_score(pipe,X_train,y_train,cv=5,scoring='accuracy').mean()


0.6437358661239257

In [ ]:
#hyper parameter tuning
# improving performance by usingf hyper parameter tuning
paras={
    'trf5__max_depth':[1,2,3,4,5,None]
}
from sklearn.model_selection import GridSearchCV

In [ ]:

# grid=GridSearchCV(pipe,paras,cv=5,scoring='accuracy')
# grid.fit(X_train,y_train)

In [ ]:
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))

In [ ]:
####production code
pipe=pickle.load(open('pipe.pkl','rb'))

In [ ]:
test_input2 = np.array([2 ,'male' ,31.0 ,0 ,0 ,10.5,'S'],dtype=object).reshape(1,7)

In [ ]:
pipe.predict(test_input2)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


array([0])